# 
Application Level Scheduling

RADICAL-Pilot (RP) by default uses its internal scheduler to efficiently place tasks on the available cluster resources.  Some use cases however require more fine grained and/or explicit control over task placement.  RP supports that functionality with *__application level scheduling__*.  In that case the pilot will report about the available nodes and resources, but will leave it to the application to assign resources to the tasks.  A number of API functions are provided to simplify the development of such application level schedulers, and this tutorial will demonstrate their use.

<div class="alert alert-info">

__Note:__ At the moment it is not possible to mix application level scheduling and RP's internal scheduling in the same session.
__Note:__ Application level scheduling is only supported for executable tasks, RAPTOR tasks will ignore any related settings.

</div>


## Notation

The following terms will be used throughout this tutorial:

  - *__task:__* an exectable piece of work comprised of one or more processes, all running the same executable on a dedicated set of resources.
  - *__rank:__* one of theprocesses which comprise a running task.  The term _rank_ is frequently used for MPI applications, but we use it generically for any task which uses multiprocessing.
  - *__slot:__* the set of resources which are assigned to a single _rank_, i.e., to a single task process. Note that each _rank_ can utilize multiple cores and/or GPUs, usually by support of libraries and frameworks such as OpenMP, CUDA, OpenCL etc.
  - *__occupation:__* the portion of a resource assigned to a _rank_.  For example, two ranks could share a GPU, and then each of the ranks would get `occupation=0.5` assigned for that GPU.



## Pilot Resources

We will first start a normal RP session and submit a pilot.  We then wait for the pilot to become active and inspect its resources by retrieving its nodelist which at that point will be available.

In [1]:
import radical.pilot as rp
import pprint

In [2]:
session = rp.Session()

pmgr = rp.PilotManager(session)
tmgr = rp.TaskManager(session)

pilot = pmgr.submit_pilots(rp.PilotDescription(
    {'resource': 'local.localhost',
     'runtime' : 60,
     'nodes'   : 4}))

tmgr.add_pilots(pilot)
pilot.wait([rp.PMGR_ACTIVE, rp.FAILED])

assert pilot.state == rp.PMGR_ACTIVE

nodelist = pilot.nodelist
print('got %d nodes' % len(nodelist.nodes))

new session: [rp.session.thinkie.merzky.020127.0000]                           \
[tcp://10.0.0.39:10001]                                          ok
create pilot manager                                                          ok
create task manager                                                           ok
submit 1 pilot(s)
                        oklhost           4 nodes


got 4 nodes


The nodelist (type: `rp.NodeList`) has the following attributes:

  - `uniform`: boolean, indicates if the nodes have identical resources
  - `cores_per_node`, `gpus_per_node`, `mem_per_node`, `lfs_per_node`: amount of resources per node.  Those attributes will be `None` for non-uniform nodelists.
  - `nodes`: the actual list of nodes.

Let's inspect one of the nodes (`nodeslist.nodes[0]`).  A node in the nodelist has the type `rp.NodeResource` with the following attributes:

  - `index`: unique node identifier used within RP
  - `name`: node name (does not need to be unique!)
  - `mem`: available memory (in MB)
  - `lfs`: available disk storage (in MB)
  - `cores`: available CPU cores and their occupation
  - `gpus`: available GPUs and their occupation.

The core and gpu information are constructed of an integer (the resource index) and a float (the resource occupation where `0.0` is *not used* and `1.0` is *fully used*).

In [3]:
# inspect one node
print('#nodes: ', len(nodelist.nodes))

node = nodelist.nodes[0]
pprint.pprint(node.as_dict())


#nodes:  4
{'cores': [{'index': 0, 'occupation': 0.0},
           {'index': 1, 'occupation': 0.0},
           {'index': 2, 'occupation': 0.0},
           {'index': 3, 'occupation': 0.0},
           {'index': 4, 'occupation': 0.0},
           {'index': 5, 'occupation': 0.0},
           {'index': 6, 'occupation': 0.0},
           {'index': 7, 'occupation': 0.0},
           {'index': 8, 'occupation': 0.0},
           {'index': 9, 'occupation': 0.0},
           {'index': 10, 'occupation': 0.0},
           {'index': 11, 'occupation': 0.0}],
 'gpus': [{'index': 0, 'occupation': 0.0}],
 'index': 0,
 'lfs': 1000000,
 'mem': 65536,
 'name': 'localhost'}


The `nodelist` class exposed the method `find_slots(rr: RankRequirements, n_slots: int=1)` which will return a set of resources, one for each task rank. Let us disect what this means: 

  - a _rank_ is a process which is part of the set of processes which comprise a task.  The term _rank_ is frequently used for MPI applications, but we use it generically for any task which uses multiprocessing.   
  - a `slot` is defined as a set of resources which are assigned to a single _rank_, i.e., to a single task process. Note that each _rank_ can utilize multiple cores and/or GPUs, usually by support of libraries and frameworks such as OpenMP, CUDA, OpenCL etc.
  - `occupation` is defined as the portion of a resource assigned to a _rank_.  For example, two ranks could share a GPU, and then each of the ranks would get `occupation=0.5` assigned for that GPU.

The slot for the simplest possible task (in RP) would just allocate one core - say core with index `3` on the first node:


In [4]:
slot_1 = rp.Slot(node_index=0, node_name='localhost', cores=[3])
pprint.pprint(slot_1.as_dict())

{'cores': [{'index': 3, 'occupation': 1.0}],
 'gpus': [],
 'lfs': 0,
 'mem': 0,
 'node_index': 0,
 'node_name': 'localhost',
 'version': 1}


A simpler way to obtain that slot is to let the node's `NodeResource` find it for you - that will though return the first available core, not number 3 as before: 

In [5]:
rr = rp.RankRequirements(n_cores=1)
slot_2 = node.find_slot(rr)
pprint.pprint(slot_2.as_dict())

{'cores': [{'index': 0, 'occupation': 1.0}],
 'gpus': [],
 'lfs': 0,
 'mem': 0,
 'node_index': 0,
 'node_name': 'localhost',
 'version': 1}


If we now check the node, we will see that the resource occupation of the first core changed.

In [6]:
pprint.pprint(node.as_dict())

{'cores': [{'index': 0, 'occupation': 1.0},
           {'index': 1, 'occupation': 0.0},
           {'index': 2, 'occupation': 0.0},
           {'index': 3, 'occupation': 0.0},
           {'index': 4, 'occupation': 0.0},
           {'index': 5, 'occupation': 0.0},
           {'index': 6, 'occupation': 0.0},
           {'index': 7, 'occupation': 0.0},
           {'index': 8, 'occupation': 0.0},
           {'index': 9, 'occupation': 0.0},
           {'index': 10, 'occupation': 0.0},
           {'index': 11, 'occupation': 0.0}],
 'gpus': [{'index': 0, 'occupation': 0.0}],
 'index': 0,
 'lfs': 1000000,
 'mem': 65536,
 'name': 'localhost'}


We can also register our manually created slot as occupied so that later calls to `find_slot` will take that information into account (after use, a slot should be deallocated again).

In [7]:
node.allocate_slot(slot_1)
pprint.pprint(node.as_dict())

{'cores': [{'index': 0, 'occupation': 1.0},
           {'index': 1, 'occupation': 0.0},
           {'index': 2, 'occupation': 0.0},
           {'index': 3, 'occupation': 1.0},
           {'index': 4, 'occupation': 0.0},
           {'index': 5, 'occupation': 0.0},
           {'index': 6, 'occupation': 0.0},
           {'index': 7, 'occupation': 0.0},
           {'index': 8, 'occupation': 0.0},
           {'index': 9, 'occupation': 0.0},
           {'index': 10, 'occupation': 0.0},
           {'index': 11, 'occupation': 0.0}],
 'gpus': [{'index': 0, 'occupation': 0.0}],
 'index': 0,
 'lfs': 1000000,
 'mem': 65536,
 'name': 'localhost'}


To allocate a _set_ of slots, for example for a multi-rank task, RP can search the node list itself for available resources.  That search might return slots which are distributed across all nodes.  For example, the call below will allocate the resources for 4 ranks where each rank uses 4 cores and half a GPU (2 ranks can share one GPU).  As the GPU is the limiting resource in this scenario, we will be able to place at most 2 ranks per node:

In [8]:
rr = rp.RankRequirements(n_cores=4, n_gpus=1, gpu_occupation=0.5)
slots = nodelist.find_slots(rr, n_slots=4)
pprint.pprint(slots)

[Slot: {'cores': [RO: 1:1.00, RO: 2:1.00, RO: 4:1.00, RO: 5:1.00], 'gpus': [RO: 0:0.50], 'lfs': 0, 'mem': 0, 'node_index': 0, 'node_name': 'localhost', 'version': 1},
 Slot: {'cores': [RO: 6:1.00, RO: 7:1.00, RO: 8:1.00, RO: 9:1.00], 'gpus': [RO: 0:0.50], 'lfs': 0, 'mem': 0, 'node_index': 0, 'node_name': 'localhost', 'version': 1},
 Slot: {'cores': [RO: 0:1.00, RO: 1:1.00, RO: 2:1.00, RO: 3:1.00], 'gpus': [RO: 0:0.50], 'lfs': 0, 'mem': 0, 'node_index': 1, 'node_name': 'localhost', 'version': 1},
 Slot: {'cores': [RO: 4:1.00, RO: 5:1.00, RO: 6:1.00, RO: 7:1.00], 'gpus': [RO: 0:0.50], 'lfs': 0, 'mem': 0, 'node_index': 1, 'node_name': 'localhost', 'version': 1}]


## Application Level Scheduling

With the above tools, the simplest implementation of an application level scheduler would be:


In [9]:

def run_tasks(tmgr, nodelist, tds):
    '''
    tmgr    : task manager which handles execution of tasks
    nodelist: resources to be used for task execution
    tds     : list of rp.TaskDescriptions - list of tasks to run
    '''

    running_tasks   = dict()
    completed_tasks = list()
    
    while tds:

        # find slots for all task descriptions
        allocated = list()
        not_allocated = list()

        for td in tds:
            rr = rp.RankRequirements(n_cores=td.cores_per_rank, n_gpus=td.gpus_per_rank, 
                                     mem=td.mem_per_rank,  lfs=td.lfs_per_rank)
            slots = nodelist.find_slots(rr, n_slots=td.ranks)
            if slots:
                # this task can be submitted
                td.slots = slots
                allocated.append(td)
            else:
                # this task has to be retries later on
                not_allocated.append(td)

        # submit all tasks for which resources were found
        submitted = tmgr.submit_tasks(allocated)
        for task in submitted:
            running_tasks[task.uid] = task
            tasks[task.uid] = task
            print('submitted %d' % task.uid)

        if not not_allocated:

            # all tasks are submitted - wait for completion
            tmgr.wait_tasks()
            break

        else:

            # could not submit all tasks - wait for resources to become available 
            # on and attempt to schedule the remaining tasks in the next iteration
            tds = not_allocated
            
            while True:
                uids   = list(running_tasks.keys())
                states = tmgr.wait_tasks(state=rp.FINAL, uids=uids, timeout=5.0)
                                
                # if some task completed, we should have new resources
                resources = False
                for uid, state in zip(uids, states):
                    if state in rp.FINAL:
                        compleded_tasks.append(running_tasks[uid])
                        pilot.nodelist.release_slots(task.description.slots)
                        del running_tasks[uid]
                        resources = True

                # if we got any free resources, try to schedule more tasks
                if resources:
                    break

tds = list()
for i in range(5):
    td = {'executable'     : 'sleep', 
          'arguments'      : [i + 5],
          'ranks'          : i + 1, 
          'cores_per_rank' : i + 1,
          'slots'          : slots}
    tds.append(rp.TaskDescription(td))

run_tasks(tmgr, pilot.nodelist, tds)

submit: ########################################################################


NameError: name 'tasks' is not defined